# EME with MEOW

Some components are more efficiently modeled with Eigenmode Expansion.

Gdsfactory provides a plugin for MEOW to efficiently extract component S-parameters through EME.

Currently the component needs to specifically have a single "o1" port facing west, and a single "o2" port facing east, like this taper:

In [ ]:
import gdsfactory as gf
import matplotlib.pyplot as plt
import numpy as np
from gdsfactory.generic_tech import get_generic_pdk

from gplugins.meow import MEOW

gf.config.rich_output()
PDK = get_generic_pdk()
PDK.activate()

LAYER_STACK = PDK.layer_stack
c = gf.components.taper_cross_section_sine()
c.plot()

You also need to explicitly provide a LayerStack to define cross-sections, for instance the generic one:

In [ ]:
layer_stack = LAYER_STACK

filtered_layer_stack = gf.technology.LayerStack(
    layers={
        k: layer_stack.layers[k]
        for k in (
            "slab90",
            "core",
            "box",
            "clad",
        )
    }
)

Since you need to make sure that your entire LayerStack has e.g. material information for all present layers, it is safer to only keep the layers that you need for your simulation:

The EME simulator can be instantiated with only these two elements, alongside parameters:

In [ ]:
eme = MEOW(
    component=c, layer_stack=filtered_layer_stack, wavelength=1.55, overwrite=True
)

Plotting functions allow you to check your simulation:

In [ ]:
eme.plot_structure()

The cross-section themselves:

In [ ]:
eme.plot_cross_section(xs_num=0)

In [ ]:
eme.plot_cross_section(xs_num=-1)

And the modes (after calculating them):

In [ ]:
eme.plot_mode(xs_num=0, mode_num=0)

In [ ]:
eme.plot_mode(xs_num=-1, mode_num=0)

The S-parameters can be calculated, and are returned in the same format as for the FDTD solvers (the original MEOW S-parameter results S and port_names are saved as attributes):

In [ ]:
sp = eme.compute_sparameters()

In [ ]:
print(np.abs(sp["o1@0,o2@0"]) ** 2)

In [ ]:
print(eme.port_map)
eme.plot_s_params()

As you can see most light stays on the fundamental TE mode

## Sweep EME length

Lets sweep the length of the taper.

In [ ]:
layer_stack = LAYER_STACK

filtered_layer_stack = gf.technology.LayerStack(
    layers={
        k: layer_stack.layers[k]
        for k in (
            "slab90",
            "core",
            "box",
            "clad",
        )
    }
)

c = gf.components.taper(width1=0.5, width2=2, length=10.0)
c.plot()

Lets do a convergence tests on the `cell_length` parameter. This depends a lot on the structure.

In [ ]:
trans = []
cells_lengths = [0.1, 0.25, 0.5, 0.75, 1]

for cell_length in cells_lengths:
    m = MEOW(
        component=c,
        layer_stack=filtered_layer_stack,
        wavelength=1.55,
        overwrite=True,
        spacing_y=-3,
        cell_length=cell_length,
    )
    sp = m.compute_sparameters()
    te0_trans = np.abs(sp["o1@0,o2@0"]) ** 2
    trans.append(te0_trans)

plt.plot(cells_lengths, trans, ".-")
plt.title("10um taper, resx = resy = 100, num_modes = 4")
plt.xlabel("Cell length (um)")
plt.ylabel("TE0 transmission")

In [ ]:
eme = MEOW(
    component=c,
    layer_stack=filtered_layer_stack,
    wavelength=1.55,
    overwrite=True,
    spacing_y=-3,
    cell_length=0.25,
)

In [ ]:
eme.plot_cross_section(xs_num=0)

In [ ]:
eme.plot_mode(xs_num=0, mode_num=0)

In [ ]:
eme.plot_cross_section(xs_num=-1)

In [ ]:
eme.plot_mode(xs_num=-1, mode_num=0)

In [ ]:
sp = eme.compute_sparameters()

In [ ]:
print(eme.port_map)
eme.plot_s_params()

In [ ]:
T = np.abs(sp["o1@0,o2@0"]) ** 2
T

In [ ]:
np.abs(sp["o1@0,o2@2"]) ** 2

In [ ]:
lengths = np.array([1, 2, 3, 5, 10, 20])
T = np.zeros_like(lengths, dtype=float)

In [ ]:
for length in lengths:
    c = gf.components.taper(width1=0.5, width2=2, length=length)
    c.plot()

In [ ]:
for i, length in enumerate(lengths):
    print(f"{length=}")
    c = gf.components.taper(width1=0.5, width2=2, length=length)
    eme = MEOW(
        component=c,
        layer_stack=filtered_layer_stack,
        wavelength=1.55,
        overwrite=True,
        spacing_y=-3,
        cell_length=0.25,
    )
    sp = eme.compute_sparameters()
    T[i] = np.abs(sp["o1@0,o2@0"]) ** 2

In [ ]:
plt.plot(lengths, T, marker="o")
plt.ylim(0.6, 1.0)
plt.title("Fundamental mode transmission")
plt.ylabel("Transmission")
plt.xlabel("taper length (um)")
plt.grid(True)
plt.show()

In [ ]:
eme.plot_s_params()